In [1]:
%%capture pip_output
!pip3 install -r requirements.txt

%load_ext dotenv
%dotenv

In [2]:
# Confirm that the install worked and we have an API key
if pip_output.stderr:
    print("An error occurred during PIP install or dotenv load:\n", pip_output.stderr)
else:
    print("PIP Install successful, no errors reported.")

import os
api_key = os.getenv('OPENAI_API_KEY')
print("API Key Loaded:", api_key is not None)
print("API key starts with:", api_key[:5])

PIP Install successful, no errors reported.
API Key Loaded: True
API key starts with: sk-7q


In [3]:
from llama_index.tools import FunctionTool
from llama_index.llms import OpenAI
from llama_index.agent import ReActAgent


# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

# initialize llm
llm = OpenAI(model="gpt-3.5-turbo-0613")

# initialize ReAct agent
agent = ReActAgent.from_tools([multiply_tool], llm=llm, verbose=True)

In [5]:
# Simple chat/response
agent.chat("What's 3*7?")

Thought: I can use the multiply tool to calculate the product of 3 and 7.
Action: multiply
Action Input: {'a': 3, 'b': 7}
Observation: 21
Thought: I have the answer to the question.
Response: 3 multiplied by 7 is equal to 21.


AgentChatResponse(response='3 multiplied by 7 is equal to 21.', sources=[ToolOutput(content='21', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 3, 'b': 7}}, raw_output=21)], source_nodes=[])

In [6]:
# Async chat

response = agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

Thought: I can use the multiply tool to find the product of 121 and 2.
Action: multiply
Action Input: {'a': 121, 'b': 2}
Observation: 242
 Once upon a time, there was a group of 242 mice living in a cozy little burrow. These mice were known for their intelligence and resourcefulness. They worked together as a team to gather food, build their nests, and protect each other from predators. Every day, they would venture out into the fields, searching for tasty grains and seeds to bring back to their burrow. With their combined efforts, they were able to gather enough food to sustain themselves through the harsh winter months. The mice lived harmoniously, sharing their resources and looking out for one another. They were a close-knit community, always ready to lend a helping paw to a fellow mouse in need. And so, the group of 242 mice thrived, living happily ever after in their cozy burrow.